In [2]:
pip install -U classiq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.6/541.6 kB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 2.7 MB/s eta 0:00:0000:010:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 48.2 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: Pyomo
    Found existing installation: Pyomo 6.8.2
    Uninstalling Pyomo-6.8.2:
      Successfully uninstalled Pyomo-6.8.2
  Attempting uninstall: pyda

In [3]:
import classiq
classiq.authenticate()

Your user code: GKKX-TZZL
If a browser doesn't automatically open, please visit this URL from any trusted device: https://auth.classiq.io/activate?user_code=GKKX-TZZL


In [9]:
from classiq import CReal

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from PIL import Image
import numpy as np
from classiq import qfunc, synthesize, execute
from classiq import Pauli, RX, RY, RZ

class ClassiqQuantumStyleTransfer:
    def __init__(self, content_path, style_path, num_qubits=4, max_size=400, shots=1000):
        """
        Quantum-Enhanced Neural Style Transfer using Classiq
        
        Args:
            content_path (str): Path to content image
            style_path (str): Path to style image
            num_qubits (int): Number of qubits for quantum feature mapping
            max_size (int): Maximum image dimension
            shots (int): Number of measurements for quantum circuit execution
        """
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.num_qubits = num_qubits
        self.shots = shots
        
        # Image Preprocessing
        self.transform = transforms.Compose([
            transforms.Resize((max_size, max_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                 std=[0.229, 0.224, 0.225])
        ])
        
        # Load VGG-19 for feature extraction
        self.vgg = models.vgg19(weights=models.VGG19_Weights.DEFAULT).features
        for param in self.vgg.parameters():
            param.requires_grad_(False)
        self.vgg.to(self.device)
        
        # Load and preprocess images
        self.content_image = self._load_image(content_path)
        self.style_image = self._load_image(style_path)

    def _load_image(self, image_path):
        """Load and preprocess a single image"""
        image = Image.open(image_path).convert("RGB")
        return self.transform(image).unsqueeze(0).to(self.device)

    def quantum_feature_transformation(self, features):
        """
        Transform classical features using quantum computation
        
        Args:
            features (torch.Tensor): Classical image features
        
        Returns:
            np.ndarray: Quantum feature representation
        """
        # Convert tensor to numpy and flatten
        features_np = features.detach().cpu().numpy().flatten()
        
        # Normalize features to [0, π] range for rotation angles
        features_np = (features_np - features_np.min()) / (features_np.max() - features_np.min() + 1e-8) * np.pi
        
        # Select subset of features if too many
        if len(features_np) > self.num_qubits * 3:  # We need at least 3 features per qubit
            # Take evenly spaced samples
            indices = np.linspace(0, len(features_np)-1, self.num_qubits * 3).astype(int)
            selected_features = features_np[indices]
        else:
            # If we don't have enough features, repeat them
            selected_features = np.tile(features_np, (self.num_qubits * 3 + len(features_np) - 1) // len(features_np))[:self.num_qubits * 3]
        
        try:
            # Define quantum circuit function for this specific feature set
            @qfunc
            def main():  # CHANGED: renamed from quantum_circuit to main
                # Feature encoding using rotation gates
                for i in range(self.num_qubits):
                    # Apply rotation gates for amplitude encoding
                    RX(selected_features[i % len(selected_features)])(i)
                    RY(selected_features[(i+1) % len(selected_features)])(i)
                    RZ(selected_features[(i+2) % len(selected_features)])(i)
                
                # Entanglement layer - create quantum correlations
                for i in range(self.num_qubits-1):
                    Pauli.CNOT(i, i+1)
                
                # Second rotation layer for expressivity
                for i in range(self.num_qubits):
                    RY(selected_features[(i+3) % len(selected_features)])(i)
                
                # Final entanglement
                for i in range(self.num_qubits-1, 0, -1):
                    Pauli.CNOT(i, i-1)
            
            # Synthesize and execute the circuit
            circuit = synthesize(main)  # CHANGED: use main instead of quantum_circuit
            
            # Execute quantum circuit and measure all qubits
            result = execute(circuit, measure_qubits=list(range(self.num_qubits)), shots=self.shots)
            
            # Extract measurement probabilities
            counts = result.get_counts()
            
            # Convert to quantum feature vector (probability distribution)
            qubits = self.num_qubits
            quantum_features = np.zeros(2**qubits)
            
            for bitstring, count in counts.items():
                # Convert bitstring to index
                index = int(bitstring, 2)
                quantum_features[index] = count / self.shots
                
            print("Quantum circuit executed successfully")
        
        except Exception as e:
            print(f"Quantum circuit execution error: {e}")
            # Fallback to classical approach if quantum execution fails
            quantum_features = np.zeros(2**self.num_qubits)
            # Use a more sophisticated fallback based on feature values
            for i in range(min(len(selected_features), 2**self.num_qubits)):
                idx = i % 2**self.num_qubits
                quantum_features[idx] += abs(np.sin(selected_features[i])) / len(selected_features)
            
            # Normalize to make it a proper probability distribution
            if np.sum(quantum_features) > 0:
                quantum_features /= np.sum(quantum_features)
            else:
                # If all values are zero, set a single state
                quantum_features[0] = 1.0
            
        return quantum_features

    def compute_quantum_loss(self, generated_features, style_features):
        """
        Compute quantum-enhanced loss using quantum feature vectors
        
        Args:
            generated_features (torch.Tensor): Features of generated image
            style_features (torch.Tensor): Features of style image
        
        Returns:
            float: Quantum-enhanced loss
        """
        # Get quantum feature vectors from quantum circuit execution
        gen_quantum_features = self.quantum_feature_transformation(generated_features)
        style_quantum_features = self.quantum_feature_transformation(style_features)
        
        # Compute quantum fidelity (similarity between quantum states)
        # Using classical fidelity formula for probability distributions
        fidelity = np.sum(np.sqrt(gen_quantum_features * style_quantum_features))
        
        # Loss is inverse of fidelity
        return 1.0 - fidelity

    def transfer_style(self, steps=500, learning_rate=0.003):
        """
        Perform neural style transfer with quantum-enhanced features
        
        Returns:
            PIL.Image: Stylized output image
        """
        # Initialize generated image
        generated_image = self.content_image.clone().requires_grad_(True)
        optimizer = optim.Adam([generated_image], lr=learning_rate)

        for step in range(steps):
            optimizer.zero_grad()
            
            # Extract features
            gen_features = self._get_features(generated_image)
            content_features = self._get_features(self.content_image)
            style_features = self._get_features(self.style_image)
            
            # Compute classical losses
            content_loss = torch.mean((gen_features["conv4_2"] - 
                                       content_features["conv4_2"]) ** 2)
            
            style_loss = torch.mean((self.gram_matrix(gen_features["conv1_1"]) - 
                                     self.gram_matrix(style_features["conv1_1"])) ** 2)
            
            # Compute quantum-enhanced loss
            quantum_loss = self.compute_quantum_loss(
                gen_features["conv4_1"].flatten()[:100],  # Take subset of features for efficiency
                style_features["conv4_1"].flatten()[:100]
            )
            
            # Combined loss
            total_loss = (
                1.0 * content_loss +    # Content preservation
                1e4 * style_loss +      # Style transfer
                0.1 * torch.tensor(quantum_loss, device=self.device, requires_grad=True)  # Quantum feature similarity
            )
            
            # Backpropagate and optimize
            total_loss.backward()
            optimizer.step()
            
            # Print progress
            if step % 10 == 0:
                print(f"Step {step}/{steps}")
                print(f"Content Loss: {content_loss.item():.4f}")
                print(f"Style Loss: {style_loss.item():.4f}")
                print(f"Quantum Loss: {quantum_loss:.4f}")
                print(f"Total Loss: {total_loss.item():.4f}\n")
                
                # Optional: Save intermediate result
                if step % 100 == 0:
                    intermediate = transforms.ToPILImage()(
                        generated_image.squeeze().detach().cpu().clamp(0, 1)
                    )
                    intermediate.save(f"intermediate_step_{step}.jpg")

        # Convert and return final image
        output_image = transforms.ToPILImage()(
            generated_image.squeeze().detach().cpu().clamp(0, 1)
        )
        return output_image

    def _get_features(self, image):
        """Extract features from specific VGG-19 layers"""
        features = {}
        x = image
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in ["0", "5", "10", "19", "21", "28"]:
                layer_name = {
                    "0": "conv1_1", "5": "conv2_1", "10": "conv3_1", 
                    "19": "conv4_1", "21": "conv4_2", "28": "conv5_1"
                }[name]
                features[layer_name] = x
        return features

    def gram_matrix(self, tensor):
        """Compute Gram matrix for style loss"""
        _, d, h, w = tensor.shape
        tensor = tensor.view(d, h * w)
        return torch.mm(tensor, tensor.t())

def main():
    # Example usage
    content_path = '/kaggle/input/flower/Balloon-Flower-Growing-in-the-Garden.jpg'  # Replace with your content image path
    style_path = '/kaggle/input/starynight1/starynight1.jpg'      # Replace with your style image path
    
    style_transfer = ClassiqQuantumStyleTransfer(content_path, style_path, num_qubits=6, shots=1000)
    output = style_transfer.transfer_style(steps=300)
    output.save("quantum_style_transfer_output.jpg")

if __name__ == "__main__":
    main()

Quantum circuit execution error: Argument '0.0' to parameter 'theta' of function 'RX' has incompatible type; expected CReal
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack or open a support ticket at: https://classiq-community.freshdesk.com/support/tickets/new
Quantum circuit execution error: Argument '1.0581386' to parameter 'theta' of function 'RX' has incompatible type; expected CReal
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack or open a support ticket at: https://classiq-community.freshdesk.com/support/tickets/new
Step 0/300
Content Loss: 0.0000
Style Loss: 2325295872.0000
Quantum Loss: 0.9783
Total Loss: 23252959232000.0977

Quantum circuit execution error: Argument '0.0' to parameter 'theta' of function 'RX' has incompatible type; expected CReal
If you need further assistance, please reach out on our Community Slack channel at: https: